In [ ]:
from numpy.random import binomial
#probability 0< f <= 0.5
f = 0.3
#number of neurons
N = 10
#number of random patterns
p = 7

def generate_data():
    eta_i_mu = binomial(1,f, (N,p))
    return(eta_i_mu)
    
    

In [ ]:
# https://blog.dbrgn.ch/2013/3/26/perceptrons-in-python/
# http://machinelearningmastery.com/implement-perceptron-algorithm-scratch-python/

from random import choice
from numpy import array, dot, random


#Decision boundary
T = 0
#Robustness
K = 0

# Unit_step does not include K, the robustness. 
# A preprocessing step will ignore data too close to the boundary.
unit_step = lambda x: 0 if x < T else 1

#No dummy, because the decision boundary stays in the unit_step function
training_data = [ (array([0,0,1]), 0), (array([0,1,1]), 1), (array([1,0,1]), 1), (array([1,1,1]), 1), ]

w = random.rand(3)
errors = []
l_rate = 0.2
n = 100
for i in xrange(n):
    x, expected = choice(training_data)
    result = dot(w, x)
    if (result > T+K) or (result < T-K):
        error = expected - unit_step(result)
        errors.append(error)
    w += l_rate * error * x
    #Non-negativity constraint
    w = w[w>0] * w

for x, _ in training_data:
    result = dot(x, w)
    print("{}: {} -> {}".format(x[:2], result, unit_step(result)))